In [148]:
import numpy as np
from matplotlib import pyplot as plt
import warnings
import matplotlib.cm as cm
%matplotlib inline
import random 
import time
import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
#from scipy.sparse.linalg import lsqr

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 
from sklearn import metrics


#  Note   I was having problems with the other virtual environment so i have switched to the porject one instead of test 
# and sklearn is now working just great.

In [149]:
def variance(data, target):
    mean=data[target].sum()/len(data[target])
    xs=sum([x*x for x in data[target]])
    if len(data[target])==1:
        var=(xs-mean**2*len(data[target]))
    else:
        var=(xs-mean**2*len(data[target]))/(len(data[target])-1)
    return var

In [150]:
def split_rf(dataframe, target, min_leaf, max_features):
    n=dataframe[target].unique().sum()
    variables=dataframe.drop(target, axis=1, inplace=False) #this drops the target column from the data for the next line
    n_cols=round(len(variables.columns)*max_features) #The number of random columns we will look at
    options=variables.columns.tolist() #A list of all column names in the data set
    select=random.sample(range(len(variables.columns)), n_cols) # how to randomly sel
    colls=[]
    for i in select:
        colls.append(options[i])
    var=[variance(dataframe, target), 0,0,0, 0] # making an array to contain information about how the split is made
    if len(dataframe)>2*min_leaf: #added to speed things up
        for col in colls:
            if dataframe[col].dtypes=='int64' or 'float64':
                c=dataframe.sort_values(col) #This sorts each of the variable columns 
                u=c[col].unique()            
                for i in range(0, len(u)-1): #np.linspace(0, len(u)-1):
                    split=(u[i]+u[i+1])/2
                    l=len(dataframe)
                    s1=pd.DataFrame(dataframe[dataframe[col]<split])
                    s2=pd.DataFrame(dataframe[dataframe[col]>split])    
                    v=variance(s1, target)*len(s1)/l+variance(s2, target)*len(s2)/l # This is where I introduced a weighting, outlined above
                    if v<var[0] and len(s1)>min_leaf and len(s2)>min_leaf:
                        var=[v, col, split, s1, s2]
    if var[1]==0:
        var='leaf'
    return var

In [151]:
def rf_trees(data, target, min_leaf, max_branches, max_features):
    st=time.time()
    #print('I hope youve inputted the max_features option as a decimal percentage')
    n=0
    leaf=[]
    v, col, sp, s1, s2=split_rf(data, target, min_leaf, max_features)
    f=[[col, sp, 0]]
    d =  [[[0], s1], [[1], s2]]
    while n < max_branches:
        n += 1
        new=[]
        #print('the size of d is {}'.format(len(d)))
        for i in d:
            try:
                v, col, sp, s1, s2=split_rf(i[1], target, min_leaf, max_features)
                t=i[0].copy()
                p=i[0].copy()
                if col == 0: #The idea of this is that, from the new_split function, if the data cannot be split anymore, the col column will be empty so we know to add this to the list
                    leaf.append(i)
                    f.append([0, 0, i[0], i[1][target].mean()])
                else:
                    t.append(0)
                    p.append(1)
                    new.append([t, s1])
                    new.append([p, s2])
                    f.append([col, sp, i[0]])
            except ValueError: #This was brought in to sort a weird error where the algortithm was only finding 4 values from the list when there was five
                leaf.append(i)
                f.append([0, 0, i[0], i[1][target].mean()])
        d=new
    for j in d:
        f.append([0, 0, j[0], j[1][target].mean()])
    d.extend(leaf)
    end=time.time()
    print('The time taken for a whole tree formation, not including bootstrap of making it in the right form is:', end-st)
    return d, f

In [152]:
def bootstrap(train):
    boot=train.copy().reset_index(drop=True)
    n=len(boot)
    vals=np.random.randint(n, size=n)
    b=pd.DataFrame()
    listt=[]
    for i in vals:
        listt.append(boot.loc[i,:])
    b=pd.concat(listt, axis=1)
    b=b.T
    end4=time.time()
    return b

In [163]:
def plant(leaves):
    E=leaves
    max=len(E[-1][2])
    branch=[]
    for i in range(0, max+1):
        y=np.zeros(2**i).tolist()
        for j in E[1:]:
            if len(j[2])==i:
                n=0           
                for k in range(len(j[2])):
                    if j[2][k]==1:
                        n+=2**(len(j[2])-k-1)
                y[n]=j 
        branch.append(y)
    branch[0][0]=E[0]
    return branch

In [154]:
def random_forest(training_data, target,min_samples_leaf, max_depth, max_features, n_trees):
    trees=[]
    for i in range(n_trees):
        strf=time.time()
        data=bootstrap(training_data)
        leaves, splits=rf_trees(data, target, min_samples_leaf, max_depth, max_features)
        branch=plant(splits)
        trees.append(branch)
        endrf=time.time()
        print('The time taken for a whole tree formation:', endrf-strf)
    return trees

In [155]:
def predict(data, branch, name):
    data=data.reset_index(drop=True)
    data[name]=np.zeros(len(data))
    for j in range(len(data)):
        p=data.loc[j, :]
        if p[branch[0][0][0]]<branch[0][0][1]:
            sep=[0] 
        else:
            sep=[1]
        val='bol'
        for i in range(1, len(branch[1:])):
            n=0           
            for k in range(len(sep)):
                if sep[k]==1:
                    n+=2**(i-k-1)
            if len(branch[i][n])==4:
                val=branch[i][n][3]
                break ##Added to try and sort out code not working. If this above condition is made we have reached 
                # the leaf and need to not try and travel further down the tree!
            elif p[branch[i][n][0]]<branch[i][n][1]:
                sep.append(0)
            else:
                sep.append(1)
        if val=='bol':
            n=0           
            for k in range(len(sep)):
                if sep[k]==1:
                    n+=2**(len(sep)-k-1)
            val=branch[-1][n][3]
        data.at[j, name]=val
    return data

In [156]:
def forest_predict(data, tree):
    n=0
    start=len(data.columns)
    for i in tree:
        name='prediction_{}'.format(str(n))
        n+=1
        data=predict(data, i, name)
    preds=data.iloc[:,start:]
    data['final_predictions']=preds.T.mean().round(1).tolist()
    return data

In [157]:
def m_a_e(predictions, target):
    abso=(predictions[target]-predictions['final_predictions']).abs()
    print('Mean absolute error:', abso.mean())

In [158]:
def OneHot(df, col):
    new=df[col].unique()
    if df[col].dtypes=='object':
        for i in new:
            df[i]=np.zeros(len(df[col])).astype(int)
            df[i][df[col]==i]=1
        df.drop(col, axis=1, inplace=True)
    return df     

In [159]:
pd.set_option('display.max_columns', 10)
house=pd.read_csv('data\House_Rent_Dataset.csv')
house.drop(["Posted On", 'Area Locality', 'Floor'], axis=1, inplace=True ) #With the one-hot encoding i am doing this makes the dataframes too long
for i in house.columns:
    house=OneHot(house, i)
#print(len(house))
train=house.head(4000)
test=house.tail(746)

In [164]:
st2=time.time()
tree_2=random_forest(train, 'Rent', 3, 10 , 0.33, 1)
end2=time.time()
print('forest build time :', end2-st2)


predictions=forest_predict(test, tree_2)

m_a_e(predictions, 'Rent')

The time taken for a whole tree formation, not including bootstrap of making it in the right form is: 3.3900203704833984
branch_1 [[0.0], [['Contact Owner', 0.5, [0]], ['Mumbai', 0.5, [1]]], [['Kolkata', 0.5, [0, 0]], ['Mumbai', 0.5, [0, 1]], ['Delhi', 0.5, [1, 0]], ['Unfurnished', 0.5, [1, 1]]], [['BHK', 2.5, [0, 0, 0]], ['BHK', 2.5, [0, 0, 1]], ['Bathroom', 2.5, [0, 1, 0]], ['Size', 1575.0, [0, 1, 1]], ['Chennai', 0.5, [1, 0, 0]], ['Bachelors/Family', 0.5, [1, 0, 1]], [0, 0, [1, 1, 0], 490416.6666666667], [0, 0, [1, 1, 1], 352857.14285714284]], [['Bangalore', 0.5, [0, 0, 0, 0]], ['Bathroom', 3.5, [0, 0, 0, 1]], ['BHK', 1.5, [0, 0, 1, 0]], ['Size', 1310.0, [0, 0, 1, 1]], ['Bathroom', 1.5, [0, 1, 0, 0]], ['Size', 1792.5, [0, 1, 0, 1]], ['BHK', 1.5, [0, 1, 1, 0]], [0, 0, [0, 1, 1, 1], 264000.0], ['Size', 2554.0, [1, 0, 0, 0]], ['Unfurnished', 0.5, [1, 0, 0, 1]], ['Bathroom', 4.5, [1, 0, 1, 0]], ['Size', 2750.0, [1, 0, 1, 1]], 0.0, 0.0, 0.0, 0.0], [['Size', 687.5, [0, 0, 0, 0, 0]], ['BHK

In [161]:
def bootstrap2(train):
    boot=train.copy().reset_index(drop=True)
    n=len(boot)
    vals=np.random.randint(n, size=n)
    b=pd.DataFrame()
    listt=[]
    for i in vals:
        listt.append(boot.loc[i,:])
    b=pd.concat(listt, axis=1)
    b=b.T
    end4=time.time()
    return b
print(bootstrap2(train))

      BHK   Rent  Size  Bathroom  Super Area  ...  Bachelors  Family  \
833     3  55000   925         2           0  ...          1       0   
2958    2  25000   950         2           0  ...          0       0   
2683    1  10000   500         1           0  ...          0       0   
3121    3  17000  1480         2           1  ...          0       0   
3133    2  14000   700         2           1  ...          0       0   
...   ...    ...   ...       ...         ...  ...        ...     ...   
3343    2   9500  1010         2           1  ...          0       0   
181     1  50000  2200         3           1  ...          0       0   
3352    3  18000  1220         2           1  ...          0       0   
3724    3  17000  1250         2           1  ...          0       0   
1063    2  35000   680         2           0  ...          0       0   

      Contact Owner  Contact Agent  Contact Builder  
833               0              1                0  
2958              1        

Decision Trees are among the most comprensible machine learning algorithms out there, and yet can be used effectively in both regression and classification problems and are the backbone of the more popular Random Forest Algorithm. The Sci-Kit Learn package means that it is easy to access decision trees as a kind of black box but it is interesting to build the framework behind them from scratch and to develop an understanding of the stages behind them.